# EchoPro Python Workflow

## Import libraries and configure the Jupyter notebook

In [1]:
import matplotlib.pyplot as plt

# Python EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.computation import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget

## Set up  EchoPro for a specific survey year

### Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [2]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

A full check of the initialization file contents needs to be done!
A check of the survey year file contents needs to be done!
CPU times: user 9.34 ms, sys: 981 µs, total: 10.3 ms
Wall time: 10 ms


### Load and process input data 
* This data is stored in `survey_2019`

In [3]:
%%time 
survey_2019.load_survey_data() #file_type='biological')

CPU times: user 1.88 s, sys: 26.1 ms, total: 1.9 s
Wall time: 1.94 s


### Compute the areal biomass density
* The areal biomass density is stored in `survey_2019.bio_calc.transect_results_gdf` as `biomass_density_adult`

In [4]:
%%time
survey_2019.compute_biomass_density()

CPU times: user 1.2 s, sys: 5.72 ms, total: 1.21 s
Wall time: 1.21 s


In [5]:
survey_2019.bio_calc.transect_results_gdf.head()

,latitude,longitude,stratum_num,transect_spacing,numerical_density,numerical_density_male,numerical_density_female,numerical_density_adult,biomass_density_male,biomass_density_female,...,abundance_adult_male,abundance_adult_female,abundance_adult,biomass_female,biomass_male,biomass,biomass_adult_female,biomass_adult_male,biomass_adult,geometry
transect_num,,,,,,,,,,,,,,,,,,,,,
1,34.397267,-121.143005,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-121.14301 34.39727)
1,34.397391,-121.133196,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-121.13320 34.39739)
1,34.397435,-121.123057,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-121.12306 34.39744)
1,34.397394,-121.112871,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-121.11287 34.39739)
1,34.397437,-121.102888,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-121.10289 34.39744)


In [6]:
print(f"Total Biomass Estimate without Kriging: {1e-6*survey_2019.bio_calc.transect_results_gdf.biomass_adult.sum():.3f} kmt")

Total Biomass Estimate without Kriging: 1643.221 kmt


## Obtain Kriging Mesh Data

In [7]:
krig_mesh = survey_2019.get_kriging_mesh()

In [8]:
krig_mesh.apply_coordinate_transformation(coord_type='transect')

In [9]:
krig_mesh.apply_coordinate_transformation(coord_type='mesh')

## Perform Ordinary Kriging of biomass density

* transformed mesh points
* semi-variogram model
* areal biomass density 

### Initialize Kriging routine

In [10]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel
)

# initalize kriging routine
krig = survey_2019.get_kriging(kriging_params)

### Perform Kriging

* Generates total biomass at mesh points
* Computes the total biomass estimate of Kriging output in kmt, where kmt is $10^3$ metric tons, which is equivalent to $10^6$ kg

In [11]:
%%time
krig.run_biomass_kriging(krig_mesh)

CPU times: user 5.31 s, sys: 6.29 s, total: 11.6 s
Wall time: 8.05 s


In [12]:
krig_results = survey_2019.bio_calc.kriging_results_gdf
krig_results.head()

,centroid_latitude,centroid_longitude,fraction_cell_in_polygon,geometry,stratum_num,biomass_density_adult_mean,biomass_density_adult_var,biomass_density_adult_samplevar,cell_area_nmi2,biomass_adult
0,49.099727,-126.024144,1.0,POINT (-126.02414 49.09973),7,0.000000,0.295113,NaN,6.25,0.000000
1,49.057959,-126.024127,1.0,POINT (-126.02413 49.05796),7,0.000000,0.029303,NaN,6.25,0.000000
2,49.016196,-126.024110,1.0,POINT (-126.02411 49.01620),7,0.000000,0.264295,NaN,6.25,0.000000
3,48.974438,-126.024093,1.0,POINT (-126.02409 48.97444),7,0.000000,0.566269,NaN,6.25,0.000000
4,48.932686,-126.024076,1.0,POINT (-126.02408 48.93269),7,50128.409807,0.717068,1.234471,6.25,313302.561296


In [13]:
krig.compute_kriging_variables()

In [14]:
ds = krig.krig_bio_calc._generate_len_age_distributions()

In [15]:
ds 

<xarray.Dataset>
Dimensions:                             (stratum: 9, len_bin: 40, age_bin: 20)
Coordinates:
  * stratum                             (stratum) int64 1 2 3 4 0 5 6 7 8
  * len_bin                             (len_bin) int64 2 4 6 8 ... 74 76 78 80
  * age_bin                             (age_bin) int64 1 2 3 4 ... 17 18 19 20
Data variables: (12/20)
    num_M                               (stratum) float64 99.0 34.0 ... 14.0
    num_F                               (stratum) float64 98.0 22.0 ... 32.0
    total_weight                        (stratum) float64 236.7 ... 4.403e+03
    len_age_weight_prop_all             (stratum) float64 0.05719 ... 0.0146
    len_age_weight_prop_M               (stratum) float64 0.0285 ... 0.003847
    len_age_weight_prop_F               (stratum) float64 0.02869 ... 0.01076
    ...                                  ...
    len_age_weight_dist_all             (stratum, len_bin, age_bin) float64 0...
    len_age_weight_dist_M               (stratum, len_bin, age_bin) float64 0...
    len_age_weight_dist_F               (stratum, len_bin, age_bin) float64 0...
    len_age_weight_dist_all_normalized  (stratum, len_bin, age_bin) float64 0...
    len_age_weight_dist_M_normalized    (stratum, len_bin, age_bin) float64 0...
    len_age_weight_dist_F_normalized    (stratum, len_bin, age_bin) float64 0...

In [16]:
biomass_male, biomass_female = krig.krig_bio_calc.set_variables(ds)

In [17]:
krig_results#["biomass_adult"].sum()

,centroid_latitude,centroid_longitude,fraction_cell_in_polygon,geometry,stratum_num,biomass_density_adult_mean,biomass_density_adult_var,biomass_density_adult_samplevar,cell_area_nmi2,biomass_adult,test,abundance,sig_b,NASC
0,49.099727,-126.024144,1.000000,POINT (-126.02414 49.09973),7,0.000000,0.295113,NaN,6.250000,0.000000,0.000000,0.000000,0.004822,0.000000
1,49.057959,-126.024127,1.000000,POINT (-126.02413 49.05796),7,0.000000,0.029303,NaN,6.250000,0.000000,0.000000,0.000000,0.004822,0.000000
2,49.016196,-126.024110,1.000000,POINT (-126.02411 49.01620),7,0.000000,0.264295,NaN,6.250000,0.000000,0.000000,0.000000,0.004822,0.000000
3,48.974438,-126.024093,1.000000,POINT (-126.02409 48.97444),7,0.000000,0.566269,NaN,6.250000,0.000000,0.000000,0.000000,0.004822,0.000000
4,48.932686,-126.024076,1.000000,POINT (-126.02408 48.93269),7,50128.409807,0.717068,1.234471,6.250000,313302.561296,313302.561296,408956.191652,0.004822,1971.895253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19838,50.446324,-129.937551,0.000026,POINT (-129.93755 50.44632),7,0.000000,1.734090,NaN,0.000162,0.000000,0.000000,0.000000,0.004822,0.000000
19839,39.965355,-126.048345,0.000022,POINT (-126.04835 39.96536),5,0.000000,1.738201,NaN,0.000138,0.000000,0.000000,0.000000,0.003712,0.000000
19840,38.233143,-123.004918,0.000013,POINT (-123.00492 38.23314),5,0.000000,0.868506,NaN,0.000083,0.000000,0.000000,0.000000,0.003712,0.000000
19841,48.073189,-127.110901,0.000011,POINT (-127.11090 48.07319),5,0.000000,1.775854,NaN,0.000067,0.000000,0.000000,0.000000,0.003712,0.000000


In [18]:
biomass_male

0            0.000000
1            0.000000
2            0.000000
3            0.000000
4        95032.184188
             ...     
19838        0.000000
19839        0.000000
19840        0.000000
19841        0.000000
19842        0.000000
Length: 19843, dtype: float64

In [19]:
biomass_female

0             0.000000
1             0.000000
2             0.000000
3             0.000000
4        218270.377109
             ...      
19838         0.000000
19839         0.000000
19840         0.000000
19841         0.000000
19842         0.000000
Length: 19843, dtype: float64